In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/processed_fraud_data.csv')   # Change path if needed
df.head()

,Transaction_Amount,Transaction_Type,Account_Balance,Device_Type,IP_Address_Flag,Previous_Fraudulent_Activity,Daily_Transaction_Count,Avg_Transaction_Amount_7d,Failed_Transaction_Count_7d,Card_Type,Transaction_Distance,Authentication_Method,Is_Weekend,Fraud_Label,Year,Month,Day,Hour,Minute,Second
0,39.79,3,93213.17,0,0,0,7,437.63,3,0,883.17,0,0,0,2023,8,14,19,30,0
1,1.19,1,75725.25,1,0,0,13,478.76,4,2,2203.36,3,0,1,2023,6,7,4,1,0
2,28.96,2,1588.96,2,0,0,14,50.01,4,3,1909.29,0,0,1,2023,6,20,15,25,0
3,254.32,0,76807.20,2,0,0,8,182.48,4,3,1311.86,1,0,1,2023,12,7,0,31,0
4,31.28,3,92354.66,1,0,1,14,328.69,4,2,966.98,3,1,1,2023,11,11,23,44,0


In [ ]:
X = df.drop(['Fraud_Label'], axis=1)
y = df['Fraud_Label']

In [ ]:
import pandas as pd
!pip install sdv
from sdv.single_table import CTGANSynthesizer
from sdv.metadata import SingleTableMetadata
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import joblib
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '':
    print(f'✅ GPU is active: {device_name}')
else:
    print('❌ GPU is not detected. Go to Runtime > Change runtime type > Select GPU.')

✅ GPU is active: /device:GPU:0


In [ ]:
data_fraud = df[df['Fraud_Label'] == 1].drop(['Fraud_Label'], axis=1)
data_nonfraud = df[df['Fraud_Label'] == 0].drop(['Fraud_Label'], axis=1)

In [ ]:
from tqdm import tqdm
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(data=data_fraud)


ctgan = CTGANSynthesizer(metadata, epochs=500, verbose=True)



ctgan.fit(data_fraud)


samples_needed = 17000
synthetic_fraud = ctgan.sample(samples_needed)


synthetic_fraud['Fraud_Label'] = 1


data_fraud['Fraud_Label'] = 1
data_nonfraud['Fraud_Label'] = 0

balanced_data = pd.concat([data_nonfraud, data_fraud, synthetic_fraud], axis=0).sample(frac=1).reset_index(drop=True)


/usr/local/lib/python3.11/dist-packages/sdv/single_table/base.py:119: FutureWarning: The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.
  warnings.warn(DEPRECATION_MSG, FutureWarning)
/usr/local/lib/python3.11/dist-packages/sdv/single_table/base.py:104: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(
Gen. (-3.71) | Discrim. (-0.07): 100%|██████████| 500/500 [10:10<00:00,  1.22s/it]


In [ ]:
balanced_data.to_csv('balanced_fraud_data.csv', index=False)
print("✅ Balanced fraud dataset saved in current directory.")



✅ Balanced fraud dataset saved in current directory.


In [ ]:
from google.colab import files
files.download('balanced_fraud_data.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import numpy as np


balanced_data['IP_Address_Flag_Clean'] = balanced_data['IP_Address_Flag'].apply(
    lambda x: 1 if isinstance(x, str) and ':' in x else int(x)
)


balanced_data.drop('IP_Address_Flag', axis=1, inplace=True)
balanced_data.rename(columns={'IP_Address_Flag_Clean': 'IP_Address_Flag'}, inplace=True)

print(df['IP_Address_Flag'].value_counts())


IP_Address_Flag
0    47490
1     2510
Name: count, dtype: int64


In [ ]:

balanced_data.to_csv('cleaned_fraud_data.csv', index=False)
print("✅ Cleaned fraud dataset saved in current directory.")


from google.colab import files
files.download('cleaned_fraud_data.csv')


✅ Cleaned fraud dataset saved in current directory.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
balanced_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67000 entries, 0 to 66999
Data columns (total 20 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Transaction_Amount            67000 non-null  float64
 1   Transaction_Type              67000 non-null  int64  
 2   Account_Balance               67000 non-null  float64
 3   Device_Type                   67000 non-null  int64  
 4   Previous_Fraudulent_Activity  67000 non-null  int64  
 5   Daily_Transaction_Count       67000 non-null  int64  
 6   Avg_Transaction_Amount_7d     67000 non-null  float64
 7   Failed_Transaction_Count_7d   67000 non-null  int64  
 8   Card_Type                     67000 non-null  int64  
 9   Transaction_Distance          67000 non-null  float64
 10  Authentication_Method         67000 non-null  int64  
 11  Is_Weekend                    67000 non-null  int64  
 12  Year                          67000 non-null  int64  
 13  M

In [ ]:

fraud_count = balanced_data[balanced_data['Fraud_Label'] == 1].shape[0]
nonfraud_count = balanced_data[balanced_data['Fraud_Label'] == 0].shape[0]

print(f"✅ Number of Fraud cases: {fraud_count}")
print(f"✅ Number of Non-Fraud cases: {nonfraud_count}")


✅ Number of Fraud cases: 33067
✅ Number of Non-Fraud cases: 33933
